In [1]:
# Incluir las bibliotecas requeridas para el preprocesamiento
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, classification_report
from sklearn.calibration import calibration_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV

In [2]:
import pandas as pd

# URL del archivo CSV público en S3
url = 'https://diabetes-dvcstore.s3.us-east-1.amazonaws.com/files/md5/10/1cada3906fab160ea188043d7f9a1b'
# Carga el archivo directamente en un DataFrame de pandas
df = pd.read_csv(url)

# Ver las primeras filas del DataFrame
print(df.head())



   Diabetes_012  HighBP  HighChol   BMI  HeartDiseaseorAttack  GenHlth   Age
0           0.0     0.0       0.0  21.0                   0.0      1.0   8.0
1           0.0     1.0       1.0  25.0                   0.0      3.0  10.0
2           0.0     1.0       0.0  23.0                   0.0      2.0  11.0
3           0.0     0.0       1.0  26.0                   0.0      1.0  10.0
4           0.0     0.0       1.0  25.0                   0.0      1.0   7.0


In [3]:
# Separar las características (X) y la variable objetivo (y)
# Importar la función train_test_split
from sklearn.model_selection import train_test_split
X = df.drop(columns=['Diabetes_012']) #resto de columnas
y = df['Diabetes_012']  # La columna objetivo predecir

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(X_test)

       HighBP  HighChol   BMI  HeartDiseaseorAttack  GenHlth   Age
43070     1.0       1.0  30.0                   1.0      3.0   9.0
15906     0.0       0.0  26.0                   0.0      4.0   7.0
66360     1.0       1.0  40.0                   0.0      3.0  12.0
7029      0.0       1.0  30.0                   0.0      3.0   7.0
8053      1.0       1.0  24.0                   0.0      4.0   5.0
...       ...       ...   ...                   ...      ...   ...
69056     1.0       1.0  34.0                   0.0      3.0   6.0
30082     1.0       1.0  29.0                   0.0      2.0  10.0
3016      0.0       0.0  30.0                   0.0      3.0   5.0
13378     0.0       0.0  27.0                   0.0      4.0   6.0
60779     0.0       0.0  28.0                   0.0      3.0   9.0

[17319 rows x 6 columns]


In [4]:
pip install catboost


MODELO DE REDES NEURONALES

In [5]:
# Importar las librerías necesarias
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, recall_score

# Crear el modelo de red neuronal (MLPClassifier)
modelo_nn = MLPClassifier(hidden_layer_sizes=(300,), max_iter=600, random_state=64)
modelo_nn.fit(X_train, y_train)
y_pred = modelo_nn.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Imprimir los resultados
print(f'Precisión del modelo de redes neuronales: {accuracy * 100:.2f}%')
print(f'Recall por clase: {recall* 100:.2f}%')
print('\nReporte de clasificación:')
print(classification_report(y_test, y_pred))

# Opcional: Ver las primeras filas de las predicciones en comparación con los valores reales
comparacion = pd.DataFrame({'Real': y_test, 'Predicción': y_pred})
print(comparacion.head())


Precisión del modelo de redes neuronales: 74.53%
Recall por clase: 84.75%

Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.81      0.64      0.72      8629
         1.0       0.70      0.85      0.77      8690

    accuracy                           0.75     17319
   macro avg       0.76      0.74      0.74     17319
weighted avg       0.76      0.75      0.74     17319

       Real  Predicción
43070   1.0         1.0
15906   0.0         0.0
66360   1.0         1.0
7029    0.0         1.0
8053    0.0         1.0


MODELO DE CATBOOSTCLASSIFIER

In [6]:
# Importar las librerías necesarias

from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, recall_score

# Crear el modelo de CatBoost
modelo_cb = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, random_state=42, cat_features=[], verbose=200)
modelo_cb.fit(X_train, y_train)
y_pred = modelo_cb.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

# Calcular el recall (sensibilidad) para cada clase
recall = recall_score(y_test, y_pred)

# Imprimir los resultados
print(f'Precisión del modelo CatBoost: {accuracy * 100:.2f}%')
print('\nReporte de clasificación:')
print(classification_report(y_test, y_pred))

# Mostrar el recall individual por clase
print(f'Recall por clase: {recall* 100:.2f}%')

# Opcional: Ver las primeras filas de las predicciones en comparación con los valores reales
comparacion = pd.DataFrame({'Real': y_test, 'Predicción': y_pred})
print(comparacion.head())


0:	learn: 0.6571431	total: 58.8ms	remaining: 58.8s
200:	learn: 0.4989880	total: 3.12s	remaining: 12.4s
400:	learn: 0.4935439	total: 10.3s	remaining: 15.4s
600:	learn: 0.4894964	total: 16.5s	remaining: 11s
800:	learn: 0.4862809	total: 23.8s	remaining: 5.92s
999:	learn: 0.4836834	total: 30.8s	remaining: 0us
Precisión del modelo CatBoost: 74.66%

Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.77      0.71      0.74      8629
         1.0       0.73      0.78      0.76      8690

    accuracy                           0.75     17319
   macro avg       0.75      0.75      0.75     17319
weighted avg       0.75      0.75      0.75     17319

Recall por clase: 78.49%
       Real  Predicción
43070   1.0         1.0
15906   0.0         0.0
66360   1.0         1.0
7029    0.0         0.0
8053    0.0         0.0


MODELO XGBOOST

In [7]:
#Entrenar el modelo de XGBoost
model_XGBC = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=64)
#Evaluar el modelo
model_XGBC.fit(X_train, y_train)
# Predicciones
y_pred = model_XGBC.predict(X_test)

# Métricas
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Exactitud XGBClassifier (Accuracy): {accuracy}")
print(f"Sensibilidad XGBClassifier (Recall): {recall}")
print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))

Exactitud XGBClassifier (Accuracy): 0.749003984063745
Sensibilidad XGBClassifier (Recall): 0.7840046029919447

Informe de clasificación:
              precision    recall  f1-score   support

         0.0       0.77      0.71      0.74      8629
         1.0       0.73      0.78      0.76      8690

    accuracy                           0.75     17319
   macro avg       0.75      0.75      0.75     17319
weighted avg       0.75      0.75      0.75     17319



REGRESIÓN LOGISTÍCA

In [8]:
# Entrenar el modelo de regresión logística
LR_model= LogisticRegression(solver='liblinear', random_state=42)
# Evaluar el modelo
LR_model.fit(X_train, y_train)
# Predicciones
y_pred = LR_model.predict(X_test)

# Métricas
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Exactitud Reg Log (Accuracy): {accuracy}")
print(f"Sensibilidad Reg Log (Recall): {recall}")
print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))

Exactitud Reg Log (Accuracy): 0.7479069230325076
Sensibilidad Reg Log (Recall): 0.7665132336018412

Informe de clasificación:
              precision    recall  f1-score   support

         0.0       0.76      0.73      0.74      8629
         1.0       0.74      0.77      0.75      8690

    accuracy                           0.75     17319
   macro avg       0.75      0.75      0.75     17319
weighted avg       0.75      0.75      0.75     17319



MODELO SUPPORT VECTOR MACHINE

In [9]:
# Entrenar el modelo SVM con Kernel linear
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Predicciones
y_pred = svm_model.predict(X_test)

# Métricas
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Exactitud SVM (Accuracy): {accuracy}")
print(f"Sensibilidad SVM (Recall): {recall}")
print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))

Exactitud SVM (Accuracy): 0.7475604827068537
Sensibilidad SVM (Recall): 0.7879171461449942

Informe de clasificación:
              precision    recall  f1-score   support

         0.0       0.77      0.71      0.74      8629
         1.0       0.73      0.79      0.76      8690

    accuracy                           0.75     17319
   macro avg       0.75      0.75      0.75     17319
weighted avg       0.75      0.75      0.75     17319

